In [99]:
import pandas as pd
import numpy as np
from time import sleep
from tqdm import tqdm
import random

from openai import OpenAI
from private_data import API_KEY

In [100]:
# Loading Dataset 
df = pd.read_csv('../data/famous-birthdates.csv', sep = ' ')
df = df[['firstname', 'lastname' , 'birthDate']]


df['name'] = df['firstname'] + ' ' + df['lastname']
df.drop(columns=["firstname", "lastname"], inplace=True)

# Shuffle and resize the data
df = df.sample(frac=1).reset_index(drop=True).dropna().head(2000)

# Subject and Object labels
sub_label = 'name'
obj_label = 'birthDate'

df.head(10)

,birthDate,name
0,1906-09-01,Joaquin Balaguer
1,1973-04-18,Haile Gebrselassie
2,1968-12-08,Mike Mussina
3,1975-01-25,Tim Montgomery
4,1944-12-23,Lewis A Kaplan
5,1935-06-25,Larry Kramer
6,1953-10-25,Jon Pareles
8,1935-07-13,Jack F Kemp
9,1950-03-20,William Hurt
10,1944-02-11,Michael G Oxley


In [101]:
## Query function
def true_false_query(query):
    my_query = [ {"role": "system", "content": "Answer directly only as True or False"},
        {"role": "user", "content": query} ] 
    client = OpenAI(api_key=API_KEY)
    completion = client.chat.completions.create(
        model="gpt-4", messages=my_query
    )
    return str(completion.choices[0].message.content)

In [1]:
def random_incorrect_object(column, correct_obj):
    while 1:    # assuming all objects are not equal
        try:
            guess = random.choice(column)
            if guess != correct_obj:
                return guess
        except Exception as e:
            print(f"An error occurred: {e}, obj: {correct_obj}")
            continue

In [102]:
def create_query(subject, object):  # object is a property of the subject
    return f"{subject} was born on {object}."



def preprocess(df):
    correct_rows = []
    with tqdm(total=len(df), unit="row") as pbar:
        for index, row in df.iterrows():
            query, response = None, None
            if random.choice([True, False]):
                query = create_query(row[sub_label], row[obj_label])
                response = true_false_query(query)
                
                if "true" in response.lower():
                    correct_rows.append(row)
            else:
                incorrect_object = random_incorrect_object(df[obj_label], row[obj_label])
                query = create_query(row[sub_label], incorrect_object)
                response = true_false_query(query)
                
                if "false" in response.lower():
                    correct_rows.append(row)
            
            # print(query, "-->", response)     #DEBUG
            pbar.update(1)
        
            
    new_df = pd.DataFrame(correct_rows)
    return new_df
        
        

In [103]:
known_df = preprocess(df)
# known_df    #DEBUG

  0%|          | 1/2000 [00:01<46:23,  1.39s/row]

An error occurred: 1593, obj: 1973-04-18


  2%|▏         | 39/2000 [00:49<41:54,  1.28s/row]

An error occurred: 242, obj: 1969-05-14


  4%|▍         | 80/2000 [01:41<44:46,  1.40s/row]  

...

 98%|█████████▊| 1962/2000 [42:28<00:41,  1.09s/row]

An error occurred: 632, obj: 1956-02-06


 99%|█████████▉| 1984/2000 [42:52<00:17,  1.08s/row]

An error occurred: 404, obj: 1932-01-01


100%|██████████| 2000/2000 [43:09<00:00,  1.29s/row]


In [104]:
def analogy_testing(df):
    result = []

    for i in tqdm(range(1, len(df))):
        sub1 = df.iloc[i-1][sub_label]
        obj1 = df.iloc[i-1][obj_label]
        
        sub2 = df.iloc[i][sub_label]
        obj2 = df.iloc[i][obj_label]
        
        answer = obj2
        query_type = "true"
        
        if not random.choice([True, False]):
            obj2 = random_incorrect_object(df[obj_label], answer)
            query_type = "false"
            
            
        prompt = f"{sub1}:({obj1})::{sub2}:X; X is {obj2}"
        response = true_false_query(prompt)
        status = query_type in response.lower()
        result.append({'Prompt': prompt, 'Correct Object': answer, 'GPT-reply': response, 'Status': status})
            
        
        # print(f"Prompt: {prompt} \t|\t Correct Object: {answer} \t|\t GPT-reply: {response} \t|\t Status: {status}")    #DEBUG
    
    return result

In [105]:
result = analogy_testing(known_df)

  1%|          | 19/1837 [00:22<33:06,  1.09s/it]

An error occurred: 1827, obj: 1953-05-05
An error occurred: 1512, obj: 1953-05-05


  3%|▎         | 51/1837 [01:00<32:28,  1.09s/it]

An error occurred: 1000, obj: 1952-06-06


  6%|▌         | 110/1837 [02:04<30:06,  1.05s/it]

...

 98%|█████████▊| 1796/1837 [42:35<00:51,  1.26s/it]

An error occurred: 1688, obj: 1948-04-29


 98%|█████████▊| 1806/1837 [42:48<00:47,  1.55s/it]

An error occurred: 315, obj: 1936-11-28


100%|██████████| 1837/1837 [43:32<00:00,  1.42s/it]


In [106]:
result_df = pd.DataFrame(data=result)
result_df.to_csv('results/result: person-birthdate.csv')
result_df

,Prompt,Correct Object,GPT-reply,Status
0,Joaquin Balaguer:(1906-09-01)::Haile Gebrselas...,1973-04-18,True,True
1,Haile Gebrselassie:(1973-04-18)::Mike Mussina:...,1968-12-08,False,True
2,Mike Mussina:(1968-12-08)::Tim Montgomery:X; X...,1975-01-25,False,True
3,Tim Montgomery:(1975-01-25)::Larry Kramer:X; X...,1935-06-25,True,True
4,Larry Kramer:(1935-06-25)::Jon Pareles:X; X is...,1953-10-25,True,True
...,...,...,...,...
1832,Vincent Gigante:(1928-03-29)::Keith Reddin:X; ...,1956-07-07,False,False
1833,Keith Reddin:(1956-07-07)::Franz Liszt:X; X is...,1811-10-22,True,True
1834,Franz Liszt:(1811-10-22)::Czeslaw Milosz:X; X ...,1911-06-30,True,True
1835,Czeslaw Milosz:(1911-06-30)::William Styron:X;...,1925-06-11,False,True


In [107]:
accuracy = result_df['Status'].sum()/len(result_df)
print(f"Accuracy: {accuracy*100}%")

Accuracy: 86.17310832879696%
